Toronto boroughs

Import dependencies

In [1]:
from bs4 import BeautifulSoup 
import requests 

import lxml
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Libraries imported.')

Libraries imported.


Get the data from wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib") 


Explore the data

In [3]:
table = soup.find_all("table",{"class":"wikitable sortable"})
table = table[1]

In [4]:
df = pd.read_html(str(table), flavor='html5lib')
df = df[0]
df.head()

,CDN number,City-designated neighbourhood,Former city/borough,Neighbourhoods covered,Map,Unnamed: 5
0,129,Agincourt North,Scarborough,Agincourt and Brimwood,NaN,NaN
1,128,Agincourt South-Malvern West,Scarborough,Agincourt and Malvern,NaN,NaN
2,20,Alderwood,Etobicoke,Alderwood,NaN,NaN
3,95,Annex,Old City of Toronto,The Annex and Seaton Village,NaN,NaN
4,42,Banbury-Don Mills,North York,Don Mills,NaN,NaN


In [5]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
def get_postcode(neighbourhood, borough):
    address = str(neighbourhood)+", "+borough+',Toronto, Ontario, Canada'
    geolocator = Nominatim(user_agent="to_explorer")
    location = geolocator.geocode(address)
    #print(location)
    if location != None:
        postcode = str(location).split(',')[-2]
        #print(postcode)
        postcode = postcode.split(" ")[1]
        #print(postcode)
        if postcode != "Ontario":
            return [postcode]

In [9]:
for row in df.values:
    #print(row)
    borough = str(row[2])
    neighbourhood = str(row[1])
    #rint(borough+" "+neighbourhood)
    
    #Get latitude and longitude
    try:
        [postcode] = get_postcode(neighbourhood, borough)
        neighborhoods = neighborhoods.append({'PostalCode': postcode,
                                            'Borough': borough,
                                            'Neighborhood': neighbourhood}, ignore_index=True)
    except:
        continue

In [8]:
neighborhoods.head(12)

,PostalCode,Borough,Neighborhood
0,M1V,Scarborough,Agincourt North
1,M1S,Scarborough,Agincourt South-Malvern West
2,M8W,Etobicoke,Alderwood
3,M5T,Old City of Toronto,Annex
4,M2L,North York,Banbury-Don Mills
5,M5R,Old City of Toronto,Bay Street Corridor
6,M2K,North York,Bayview Village
7,M2M,North York,Bayview Woods-Steeles
8,M1P,Scarborough,Bendale
9,M6L,North York,Black Creek
